### Goal: Add column to `comics` table for search_path to larger image.

In [1]:
import sys
import os
import pandas as pd
import boto3

# Data storage
from sqlalchemy import create_engine  # SQL helper
import psycopg2 as psql  # PostgreSQL DBs

sys.path.append("..")

In [2]:
import keys
import data_fcns as dfc

In [19]:
pd.set_option('display.max_colwidth', -1)

Images directory:

I set up an s3 bucket: https://comrx.s3-us-west-2.amazonaws.com/covers/

In [3]:
# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret',
                               'aws_ps_flatiron.json')
secret_path_aws

aws_keys = keys.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

In [4]:
# Instantiate cursor
cur = conn.cursor()

### Execute Test Query

In [5]:
#  Count records.
query = """
    SELECT * from comic_trans;
"""

In [6]:
# Execute the query
cur.execute(query)

In [7]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [8]:
temp_df.head(3)

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG)
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG)
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG)


### Look at current contents of `comics` table

In [13]:
#  Count records.
query = """
    SELECT * from comics LIMIT 10;
"""

In [14]:
# Execute the query
cur.execute(query)

In [15]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [20]:
temp_df.head(3)

,comic_id,comic_title,img_url
0,17,1 For $1 Axe Cop Bad Guy Eart (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
1,20,1 For $1 Conan the Barbarian (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
2,22,1 For $1 Mass Effect Foundati (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
